In [53]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from statsmodels.api import OLS

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category = UserWarning, module = 'lightgbm')

In [46]:
# Regression Approach

# Import Dataset
file_path = '/Users/qinhan/Desktop/UC Berkeley/Fall 2024/STAT 256/Final Project/balanced_time_agg.csv'
balanced_data = pd.read_csv(file_path)

# Define the outcome and treatment variables
outcome = 'Mortality'
treatment = 'Treatment'

# Check unique values in the outcome variable
print(f"Unique values in '{outcome}': {balanced_data[outcome].unique()}")

# Clean the outcome variable to ensure it is binary
valid_values = [0, 1]
balanced_data = balanced_data[balanced_data[outcome].isin(valid_values)]

# Prepare the data for logistic regression
X = balanced_data[[treatment]]  # Predictor: binary treatment
y = balanced_data[outcome]  # Response: binary mortality

# Add an intercept to the logistic regression model
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X)
result = model.fit()

# Print the summary of the model
print(result.summary())

# Calculate the predicted probability of mortality for treated and untreated groups
balanced_data['Predicted_Prob'] = result.predict(X)

# Calculate the average causal effect (ACE)
# E[Y(1)] - E[Y(0)]
treated_prob = balanced_data.loc[balanced_data[treatment] == 1, 'Predicted_Prob'].mean()
untreated_prob = balanced_data.loc[balanced_data[treatment] == 0, 'Predicted_Prob'].mean()
ace = treated_prob - untreated_prob

# Output the results
print(f"\nAverage Predicted Probability for Treated (E[Y(1)]): {treated_prob:.4f}")
print(f"Average Predicted Probability for Untreated (E[Y(0)]): {untreated_prob:.4f}")
print(f"Average Causal Effect (ACE): {ace:.4f}")

Unique values in 'Mortality': [0. 1.]
Optimization terminated successfully.
         Current function value: 0.623480
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:              Mortality   No. Observations:                 2263
Model:                          Logit   Df Residuals:                     2261
Method:                           MLE   Df Model:                            1
Date:                Wed, 27 Nov 2024   Pseudo R-squ.:                 0.06052
Time:                        16:57:13   Log-Likelihood:                -1410.9
converged:                       True   LL-Null:                       -1501.8
Covariance Type:            nonrobust   LLR p-value:                 1.981e-41
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1717      0.072    -16.348      0.000      -1.312    

In [40]:
# G formula
# Load the dataset
file_path = '/Users/qinhan/Desktop/UC Berkeley/Fall 2024/STAT 256/Final Project/balanced_time_agg.csv'
data = pd.read_csv(file_path)

# Define variables
treatment = 'Treatment'
outcome = 'Mortality'
confounder = 'Avg_Systolic_BP'

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

# Step 1: Model E[Y | Z2, Z1, X1, X0]
outcome_model = LogisticRegression(max_iter=1000, solver='lbfgs')
outcome_model.fit(train_data[[treatment, confounder]], train_data[outcome])

# Step 2: Model P(X1 | Z1, X0) using the confounder model
confounder_model = GradientBoostingClassifier()
confounder_model.fit(train_data[[treatment]], train_data[confounder].astype('int'))

# Step 3: Simulate potential outcomes under treatment scenarios
def simulate_potential_outcomes(data, treatment_value):
    simulated_data = data.copy()
    simulated_data[treatment] = treatment_value
    # Predict confounder X1 (Avg_Systolic_BP) for the given treatment
    simulated_data[confounder] = confounder_model.predict(data[[treatment]])
    # Predict the outcome based on the treatment and simulated confounder
    simulated_data['Predicted_Outcome'] = outcome_model.predict_proba(
        simulated_data[[treatment, confounder]]
    )[:, 1]
    return simulated_data['Predicted_Outcome'].mean()

# Compute expected outcomes under treatment and no treatment
expected_outcome_treated = simulate_potential_outcomes(test_data, treatment_value=1)
expected_outcome_untreated = simulate_potential_outcomes(test_data, treatment_value=0)

# Calculate the Average Causal Effect (ACE)
ace = expected_outcome_treated - expected_outcome_untreated

# Output results
print(f"Expected Outcome (Treated): {expected_outcome_treated:.4f}")
print(f"Expected Outcome (Untreated): {expected_outcome_untreated:.4f}")
print(f"Average Causal Effect (ACE): {ace:.4f}")


Expected Outcome (Treated): 0.4961
Expected Outcome (Untreated): 0.2353
Average Causal Effect (ACE): 0.2608


In [54]:
# Marginal Structural Model
# Load the dataset
file_path = '/Users/qinhan/Desktop/UC Berkeley/Fall 2024/STAT 256/Final Project/balanced_time_agg.csv'
data = pd.read_csv(file_path)

# Define variables
treatment = 'Treatment'
outcome = 'Mortality'
confounder = 'Avg_Systolic_BP'

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

# Step 1: Estimate propensity scores for treatment (e(z | X))
propensity_model = LogisticRegression(max_iter=1000, solver='lbfgs')
propensity_model.fit(train_data[[confounder]], train_data[treatment])

# Predict propensity scores
train_data['Propensity_Score'] = propensity_model.predict_proba(train_data[[confounder]])[:, 1]
test_data['Propensity_Score'] = propensity_model.predict_proba(test_data[[confounder]])[:, 1]

# Step 2: Compute stabilized weights (IPWs)
train_data['Stabilized_Weight'] = np.where(
    train_data[treatment] == 1,
    1 / train_data['Propensity_Score'],
    1 / (1 - train_data['Propensity_Score'])
)

# Step 3: Fit a weighted logistic regression model (MSM)
# E[Y(Z)] = β0 + β1 * Z
msm_model = LogisticRegression(max_iter=1000, solver='lbfgs')
msm_model.fit(
    train_data[[treatment]],
    train_data[outcome],
    sample_weight=train_data['Stabilized_Weight']
)

# Extract coefficients
intercept = msm_model.intercept_[0]
treatment_effect = msm_model.coef_[0][0]

# Step 4: Compute causal effect
expected_outcome_treated = msm_model.predict_proba([[1]])[0][1]  # Predicted probability for treated
expected_outcome_untreated = msm_model.predict_proba([[0]])[0][1]  # Predicted probability for untreated
ace = expected_outcome_treated - expected_outcome_untreated

# Output results
print(f"Intercept (Baseline Mortality): {intercept:.4f}")
print(f"Treatment Effect (β1): {treatment_effect:.4f}")
print(f"Expected Outcome (Treated): {expected_outcome_treated:.4f}")
print(f"Expected Outcome (Untreated): {expected_outcome_untreated:.4f}")
print(f"Average Causal Effect (ACE): {ace:.4f}")


Intercept (Baseline Mortality): -1.1271
Treatment Effect (β1): 1.1657
Expected Outcome (Treated): 0.5096
Expected Outcome (Untreated): 0.2447
Average Causal Effect (ACE): 0.2649


In [52]:
# Structural Nested Model
# Load the dataset
file_path = '/Users/qinhan/Desktop/UC Berkeley/Fall 2024/STAT 256/Final Project/balanced_time_agg.csv'
data = pd.read_csv(file_path)

# Define variables
treatment = 'Treatment'
outcome = 'Mortality'
confounder = 'Avg_Systolic_BP'

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

# Step 1: Estimate propensity scores (P(Z_t | H_t))
propensity_model = LogisticRegression(max_iter=1000, solver='lbfgs')
propensity_model.fit(train_data[[confounder]], train_data[treatment])

# Add propensity scores to the dataset
train_data['Propensity_Score'] = propensity_model.predict_proba(train_data[[confounder]])[:, 1]

# Step 2: Calculate residualized treatment effect
# Compute residualized treatment (Z_t - P(Z_t | H_t))
train_data['Residual_Treatment'] = train_data[treatment] - train_data['Propensity_Score']

# Step 3: Regress outcome on residualized treatment (G-Estimation)
# Y = β0 + ψ_t * (Z_t - P(Z_t | H_t)) + ε
g_estimation_model = OLS(
    train_data[outcome],
    sm.add_constant(train_data['Residual_Treatment'])
)
g_estimation_result = g_estimation_model.fit()

# Extract coefficient (ψ_t)
psi_t = g_estimation_result.params['Residual_Treatment']
intercept = g_estimation_result.params['const']

# Step 4: Compute causal effect for each treatment scenario
# Simulate counterfactual outcomes
train_data['Counterfactual_Untreated'] = train_data[outcome] - psi_t * train_data['Residual_Treatment']
train_data['Counterfactual_Treated'] = train_data['Counterfactual_Untreated'] + psi_t

# Compute average potential outcomes
expected_outcome_treated = train_data['Counterfactual_Treated'].mean()
expected_outcome_untreated = train_data['Counterfactual_Untreated'].mean()

# Compute Average Causal Effect (ACE)
ace = expected_outcome_treated - expected_outcome_untreated

# Output results
print(f"Coefficient of Residualized Treatment (ψ_t): {psi_t:.4f}")
print(f"Intercept (Baseline Outcome): {intercept:.4f}")
print(f"Expected Outcome (Treated): {expected_outcome_treated:.4f}")
print(f"Expected Outcome (Untreated): {expected_outcome_untreated:.4f}")
print(f"Average Causal Effect (ACE): {ace:.4f}")


Coefficient of Residualized Treatment (ψ_t): 0.2667
Intercept (Baseline Outcome): 0.3807
Expected Outcome (Treated): 0.6473
Expected Outcome (Untreated): 0.3807
Average Causal Effect (ACE): 0.2667
